<a href="https://colab.research.google.com/github/Ibtisam-a/Project-Implementation/blob/master/4_shifted_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Shifted System** 

TU + UT = F

1. Computing  $T = WSW^{-1}$, where $S = \text{diag}(s_1, \dots, s_n)$ are the eigenvalues of $T$ and the columns of $W$ are the eigenvectors of $T$

     a) Computing the eigenpairs of $T$

     b) Computing the inverse of $W$

2. Solving the system $(T+s_i I)(\hat{U})_i = (\hat{F})_i$, where $\hat{F} = FW$, where $i=1$ to $n$, 

3. Compute solution $U = \hat{U}W^{-1}$

In [25]:
import numpy as np
from scipy.sparse import diags

#Our parameters
#number of internal nodes , unknowns, in each direction

N = 190
  # our values: 190, 380, 760, 1520, 3040, 6080 and 12160

#number of internal nodes, unknowns, in each direction
n = N-2 
#step size 
h = 1/(n+1) 

#Matrix with zeros
U = np.zeros([n,n])

# x and y arrays between 0 and 1, spaced internal points
x = np.linspace(h, 1-h, n)
y = np.linspace(h, 1-h, n)

#Internal mesh without boundaries
X, Y = np.meshgrid(x, y, indexing='ij')

#our function 
F = np.sin(2*np.pi*X)*np.sin(2*np.pi*Y)  

#Tridiagonal matrix T
diagonals = [[1.5],[-0.5],[-0.5]]
T = np.multiply(1, diags(diagonals, [0, -1, 1], shape=(n, n)).toarray())

#Compute the exact solution on several time steps to be comapred 
# 0.0000001, 0.000000025, 0.00000000625, 0.0000000015625, 0.00000000039062, 0.00000000009766
Exact_sol = np.exp(-8*(np.pi*np.pi)*(0.0000001))*np.sin(2*np.pi*X)*np.sin(2*np.pi*Y)




In [26]:
import numpy as np
import time
from scipy.sparse.linalg import spsolve

def shifted_sys(T, F):
    n = len(T)
    start_time = time.time()
    eigvals = np.empty(n)
    eigvecs = np.empty([n,n])
    eigvals, eigvecs = np.linalg.eig(T) #To get eigenvalues and eigenvectors of T
    eig_time = time.time() - start_time
    S = diags(eigvals, 0, shape=(n, n)).toarray() #Set eigenvalues as diagonal matrix
    W = eigvecs
    W_inv = W.transpose() #W is W^-1 = W^T
    inv_time = time.time() - eig_time - start_time
    F_hat = np.matmul(F,W)
    U_hat = np.zeros([n,n])
    
    for i in range(0,n):
        A = T + S[i][i]*np.eye(n)
        U_hat[:,i] = spsolve(A,F_hat[:,i])
    
    solve_time = time.time() - inv_time - eig_time - start_time
    U = np.matmul(U_hat, W_inv)
    end_time = time.time()
    U_time = end_time - solve_time - inv_time - eig_time - start_time
    total_time = end_time - start_time
    all_timess = [eig_time, inv_time, solve_time, U_time, total_time]
    return U,all_timess

In [ ]:
import numpy as np

def compute_error(T, F):   
    n = len(U)
    # The maximum difference between both solutions
    err_max_difference = 0  
    # The average difference between both solutions 
    err_ave_difference = 0

#For loop to compute the error
    for i in range(0,n):
        for j in range(0,n):
            err_ave_difference += np.absolute(Exact_sol[i][j] - U[i][j])**2
            if np.absolute(Exact_sol[i][j] - U[i][j]) > err_max_difference:
                err_max_difference = np.absolute(Exact_sol[i][j] - U[i][j])       
    err_ave_difference = (err_ave_difference * h**2)**0.5
    return err_max_difference, err_ave_difference

#Solving our system.
U, all_timess = shifted_sys(T, F)   
err_max_difference, err_ave_difference = compute_error(U, Exact_sol)

print('Time taken:', all_timess)
print('Maximum diffence:', err_max_difference, '\n', 'Average difference:', err_ave_difference)